In [187]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup 
import pandas as pd 
import re
import time    

In [188]:
# Convert money lines to probabilites 
def to_prob(money_line):
    if money_line < 0: 
        prob = -money_line / (-money_line + 100) 
    elif money_line > 0: 
        prob = 100 / (money_line + 100) 
    else: 
        prob = None 

    return (prob) 

In [189]:
url = 'https://projects.fivethirtyeight.com/2019-mlb-predictions/games/'
page_response = requests.get(url)
page_content = BeautifulSoup(page_response.text, 'html.parser')

In [190]:
rows = page_content.findAll(class_="tr")

dict_list = []

for row in rows:
    try:
        date = row.find(class_='td td-datetime').getText()
    except:
        score = None
    long_name = row.find(class_='team-name long').getText()
    short_name = row.find(class_='team-name short').getText()
    pitcher = row.find(class_='pitcher-name').getText()
    num_rating = row.find(class_='td number td-number rating-adj').getText().replace('=', '')
    win_prob = row.find(class_='td number td-number win-prob').getText()
    
    
    try:
        score = row.find(class_='td number td-number score').getText()
    except:
        score = None
    
    d = {
        'date': date[:4],
        'long_name': long_name,
        'short_name': short_name,
        'pitcher': pitcher,
        'num_rating': num_rating, 
        'win_prob': win_prob.replace('%', ''),
        'score': score
    }
    
    dict_list.append(d)
    
fte_df = pd.DataFrame(dict_list)
fte_df

,date,long_name,num_rating,pitcher,score,short_name,win_prob
0,7/19,Red Sox,1563,Price,,BOS,64
1,7/19,Orioles,1461,Means,,BAL,36
2,7/19,Rockies,1499,Freeland,,COL,35
3,7/19,Yankees,1605,Happ,,NYY,65
4,7/19,Phillies,1501,Arrieta,,PHI,49
5,7/19,Pirates,1505,Lyles,,PIT,51
6,7/19,Cardinals,1521,Wainwright,,STL,52
7,7/19,Reds,1504,Mahle,,CIN,48
8,7/19,Royals,1450,Montgomery,,KC,32
9,7/19,Indians,1580,Bieber,,CLE,68


In [191]:
# Get Page and Content 
url = 'http://www.espn.com/mlb/lines'
page_response = requests.get(url)
page_content = BeautifulSoup(page_response.text, 'html.parser')

# Pull Rows of Data
rows = page_content.findAll(class_="oddrow")
rows = rows + (page_content.findAll(class_="evenrow"))

In [192]:
# List Precursor 
dict_list = []

# Main Text Parsing of MLB Games 
for row in rows: 
    if row.findAll("td")[0].getText() == 'Westgate':
        bookee = (row.findAll("td")[0].getText())
        row = (str(row.findAll("td")[1]).replace("<td style=\"text-align:center;\">", "").replace("</td>","").split("<br/>"))
        team_1 = row[0].split(":")[0]
        team_2 = row[1].split(":")[0]
        moneyline_1 = row[0].split(":")[1].strip()
        moneyline_2 = row[1].split(":")[1].strip()    
        
        #print(bookee, team_1, moneyline_1, team_2, moneyline_2)
        
        d1 = {
        'bookee': bookee,
        'short_name': team_1,
        'moneyline': moneyline_1
        }
        
        d2 = {
        'bookee': bookee,
        'short_name': team_2,
        'moneyline': moneyline_2    
        }
         
        dict_list.append(d1)
        dict_list.append(d2)
        
        
# Create df and Features 
mlb_df = pd.DataFrame(dict_list)   
mlb_df['prob'] = [round(to_prob(int(i))*100) for i in mlb_df['moneyline']]
mlb_df['date'] = str(time.strftime("%m/%d").replace('0', ''))     
mlb_df

,bookee,moneyline,short_name,prob,date
0,Westgate,160,SD,38,7/19
1,Westgate,-170,CHC,63,7/19
2,Westgate,200,COL,33,7/19
3,Westgate,-220,NYY,69,7/19
4,Westgate,-220,BOS,69,7/19
5,Westgate,200,BAL,33,7/19
6,Westgate,105,PHI,49,7/19
7,Westgate,-115,PIT,53,7/19
8,Westgate,210,CHW,32,7/19
9,Westgate,-230,TB,70,7/19


In [193]:
# Compare
lines = len(mlb_df)



In [199]:
result = pd.merge(fte_df,
                 mlb_df,
                 on=['date', 'short_name'])

result['difference'] = result['win_prob'].astype('int64') - result['prob'].astype('int64')
result

result.sort_values(by='difference', ascending=False)

,date,long_name,num_rating,pitcher,score,short_name,win_prob,bookee,moneyline,prob,difference
27,7/19,Giants,1500,Beede,,SF,43,Westgate,160,38,5
12,7/19,White Sox,1453,Lopez,,CHW,37,Westgate,210,32,5
15,7/19,Braves,1562,Teheran,,ATL,53,Westgate,100,50,3
8,7/19,Royals,1450,Montgomery,,KC,32,Westgate,250,29,3
1,7/19,Orioles,1461,Means,,BAL,36,Westgate,200,33,3
16,7/19,Rangers,1520,Minor,,TEX,35,Westgate,200,33,2
2,7/19,Rockies,1499,Freeland,,COL,35,Westgate,200,33,2
21,7/19,Diamondbacks,1504,Clarke,,ARI,49,Westgate,115,47,2
28,7/19,Padres,1479,Lauer,5,SD,39,Westgate,160,38,1
11,7/19,Tigers,1421,Zimmermann,,DET,41,Westgate,150,40,1


In [ ]:
TODO 
Scrape 'http://www.espn.com/mlb/lines' Done
get into a table Done 
add a date column 

Parse Date for 538 games 
add a date column 

join games on date and team 
compare probabilites
